# Detecção de Doença em Folhas de Café com Regressão Logística

Este projeto tem como objetivo classificar folhas de café como saudáveis ou doentes utilizando algoritmos de Machine Learning, com foco em Regressão Logística.


In [ ]:
# 1. Importações necessárias
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import joblib

: 

In [ ]:
# 3. Configurações de caminhos
base_path = "/content/drive/MyDrive/coffee-leaf-dataset"
modelos_path = "/content/drive/MyDrive/modelos"
os.makedirs(modelos_path, exist_ok=True)

In [ ]:
# 4. Mapeamento de classes (0 = saudável, 1 = qualquer doença)
label_map = {
    "Healthy": 0,
    "Healthy_augmented": 0,
    "Ferrugem": 1,
    "Ferrugem_augmented": 1,
    "Phoma": 1,
    "Phoma_augmented": 1,
    "Mineiro": 1,
    "Mineiro_augmented": 1,
    "Pulga_Vermelha": 1,
    "Pulga_Vermelha_augmented": 1,
    "Fosforo": 1,
    "Fosforo_augmented": 1
}


In [ ]:
# 5. Função para aumentar imagens de todas as classes
def augment_images(input_folder, output_folder, class_name):
    os.makedirs(output_folder, exist_ok=True)
    img_size = (128, 128)
    rotation_angles = [15, 30, 45, 60, 90, 120, 180]
    flip_modes = ['horizontal', 'vertical', 'both']

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(input_folder, filename)
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img)

            # Rotação
            for angle in rotation_angles:
                rotated = tf.keras.preprocessing.image.random_rotation(
                    img_array,
                    rg=angle,
                    row_axis=0,
                    col_axis=1,
                    channel_axis=2,
                    fill_mode='nearest'
                )
                rotated_img = array_to_img(rotated)
                new_filename = f"{os.path.splitext(filename)[0]}_{class_name}_rot{angle}.jpg"
                rotated_img.save(os.path.join(output_folder, new_filename))

            # Flip
            for flip_mode in flip_modes:
                if flip_mode == 'horizontal':
                    flipped = np.fliplr(img_array)
                elif flip_mode == 'vertical':
                    flipped = np.flipud(img_array)
                else:  # both
                    flipped = np.flipud(np.fliplr(img_array))

                flipped_img = array_to_img(flipped)
                new_filename = f"{os.path.splitext(filename)[0]}_{class_name}_flip{flip_mode}.jpg"
                flipped_img.save(os.path.join(output_folder, new_filename))

In [ ]:
# 6. Aumentar imagens para todas as classes
for class_name in os.listdir(base_path):
    class_path = os.path.join(base_path, class_name)
    if os.path.isdir(class_path) and class_name in label_map and not class_name.endswith('_augmented'):
        output_folder = os.path.join(base_path, f"{class_name}_augmented")
        augment_images(class_path, output_folder, class_name)


In [ ]:
# 7. Função para carregar e preprocessar imagens
def load_images(path, label_map, size=(128, 128)):
    X, y = [], []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if not os.path.isdir(folder_path):
            continue

        # Determinar o nome da classe base (removendo '_augmented')
        label_folder = folder.replace('_augmented', '')
        if label_folder not in label_map:
            continue

        label = label_map[label_folder]
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            X.append(img.flatten())
            y.append(label)

    return np.array(X), np.array(y)

In [ ]:
# 8. Carregar dados
X, y = load_images(base_path, label_map)
print(f"Total de amostras: {len(X)}")
print(f"Tamanho de cada imagem (flattened): {X.shape[1]}")

# Verificar balanceamento de classes
unique, counts = np.unique(y, return_counts=True)
print("\nDistribuição das classes:")
print(f"Saudáveis (0): {counts[0]}")
print(f"Doentes (1): {counts[1]}")

In [ ]:
# 9. Normalização

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 10. Redução de dimensionalidade com PCA
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
# 11. Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# 12. Treinamento do modelo de regressão logística
model = LogisticRegression(max_iter=3000, solver='saga', random_state=42)
model.fit(X_train, y_train)

In [ ]:
# 13. Avaliação
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nAcurácia: {acc:.2%}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred, target_names=["Healthy", "Doente"]))

In [ ]:
# 14. Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Healthy", "Doente"], yticklabels=["Healthy", "Doente"])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.tight_layout()
plt.show()

In [ ]:
# 15. Avaliação treino/teste
train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train, train_pred)
print(f"Acurácia no Treino: {train_acc:.2f}")

test_pred = model.predict(X_test)
test_acc = accuracy_score(y_test, test_pred)
print(f"Acurácia no Teste: {test_acc:.2f}")

In [ ]:
# 16. Curva de Aprendizado
train_sizes, train_scores, val_scores = learning_curve(
    LogisticRegression(max_iter=3000, solver='saga'),
    X_pca, y,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy'
)

train_mean = train_scores.mean(axis=1)
val_mean = val_scores.mean(axis=1)

plt.plot(train_sizes, train_mean, label='Treino')
plt.plot(train_sizes, val_mean, label='Validação')
plt.xlabel('Tamanho do conjunto de treino')
plt.ylabel('Acurácia')
plt.title('Curva de Aprendizado')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 17. Salvar modelo, scaler e PCA
joblib.dump(model, os.path.join(modelos_path, 'modelo_logistico.pkl'))
joblib.dump(scaler, os.path.join(modelos_path, 'scaler.pkl'))
joblib.dump(pca, os.path.join(modelos_path, 'pca.pkl'))
print("\nModelo, scaler e PCA salvos com sucesso!")

In [ ]:
# 18. Função para prever uma nova imagem
def prever_imagem(imagem_path, modelo, scaler, pca, size=(128, 128)):
    img = cv2.imread(imagem_path)
    if img is None:
        raise ValueError("Imagem não encontrada ou corrompida.")

    img = cv2.resize(img, size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_flat = img.flatten().reshape(1, -1)

    img_scaled = scaler.transform(img_flat)
    img_pca = pca.transform(img_scaled)

    pred = modelo.predict(img_pca)[0]
    proba = modelo.predict_proba(img_pca)[0]

    classe = "Healthy" if pred == 0 else "Doente"
    confianca = proba[pred]

    return classe, confianca

In [ ]:
# 19. Exemplo de uso com uma imagem de teste
image_test_path = "/content/drive/MyDrive/imagem_teste/folha_exemplo.jpg"

# Carregar modelo salvo
modelo_carregado = joblib.load(os.path.join(modelos_path, 'modelo_logistico.pkl'))
scaler_carregado = joblib.load(os.path.join(modelos_path, 'scaler.pkl'))
pca_carregado = joblib.load(os.path.join(modelos_path, 'pca.pkl'))

# Fazer a previsão
try:
    classe, confianca = prever_imagem(image_test_path, modelo_carregado, scaler_carregado, pca_carregado)
    print(f"\nPrevisão para a imagem de teste:")
    print(f"Classe prevista: {classe}")
    print(f"Confiança: {confianca:.2%}")
except Exception as e:
    print(f"\nErro ao processar a imagem de teste: {e}")